    16_230109_100_kyungki_apt_시흥
 
     
      data : apt_시흥_안성_635.xls
     save : apt_siheung.csv
     display : apt_siheung.html

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_시흥_안성_635.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,시흥시,NaN,LH가온수플,조남동,663,2018-07-03,25,10,678,N
1,시흥시,NaN,LH네이처뷰,장현동,170-2,2019-09-09,25,6,620,N
2,시흥시,NaN,LH네이처포레,은행동,632,2018-07-16,29,14,1594,N
3,시흥시,NaN,LH네이처하임,조남동,720,2016-07-19,24,8,592,Y
4,시흥시,NaN,LH리벤티움,조남동,678,2017-07-04,21,12,944,N


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,시흥시,LH가온수플,조남동,663,2018-07-03,25,10,678,N
1,시흥시,LH네이처뷰,장현동,170-2,2019-09-09,25,6,620,N
2,시흥시,LH네이처포레,은행동,632,2018-07-16,29,14,1594,N
3,시흥시,LH네이처하임,조남동,720,2016-07-19,24,8,592,Y
4,시흥시,LH리벤티움,조남동,678,2017-07-04,21,12,944,N


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,시흥시,LH가온수플,조남동,663,2018-07-03,25,10,678,N,경기도 시흥시 조남동 663
1,시흥시,LH네이처뷰,장현동,170-2,2019-09-09,25,6,620,N,경기도 시흥시 장현동 170-2
2,시흥시,LH네이처포레,은행동,632,2018-07-16,29,14,1594,N,경기도 시흥시 은행동 632
3,시흥시,LH네이처하임,조남동,720,2016-07-19,24,8,592,Y,경기도 시흥시 조남동 720
4,시흥시,LH리벤티움,조남동,678,2017-07-04,21,12,944,N,경기도 시흥시 조남동 678


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 시흥시 배곧동 2483은(는) 검색에 실패했습니다
경기도 시흥시 정왕동 15342은(는) 검색에 실패했습니다
경기도 시흥시 신천동 13575은(는) 검색에 실패했습니다
경기도 시흥시 신천동 13667은(는) 검색에 실패했습니다
경기도 시흥시 정왕동 15373은(는) 검색에 실패했습니다
경기도 시흥시 포동 44605은(는) 검색에 실패했습니다
경기도 시흥시 장곡동 35186은(는) 검색에 실패했습니다
경기도 시흥시 장곡동 35582은(는) 검색에 실패했습니다
경기도 시흥시 신천동 25204은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2456은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2388은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2459은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2460은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2511은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2528은(는) 검색에 실패했습니다
경기도 시흥시 신천동 44619은(는) 검색에 실패했습니다
경기도 시흥시 미산동 산 산15-1은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2381은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2502은(는) 검색에 실패했습니다
경기도 시흥시 정왕동 14611은(는) 검색에 실패했습니다
경기도 시흥시 정왕동 14642은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2383은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2575은(는) 검색에 실패했습니다
경기도 시흥시 능곡동 44805은(는) 검색에 실패했습니다
경기도 시흥시 신천동 44646은(는) 검색에 실패했습니다
경기도 시흥시 포동 44633은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2566은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2564은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2558은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2503은(는) 검색에 실패했습니다
경기도 시흥시 배곧동 2485은(는) 검색에 

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,시흥시,LH가온수플,조남동,663,2018-07-03,25,10,678,N,경기도 시흥시 조남동 663,942863.200408787,1931018.66333105
1,시흥시,LH네이처뷰,장현동,170-2,2019-09-09,25,6,620,N,경기도 시흥시 장현동 170-2,937896.12810012,1931732.41392525
2,시흥시,LH네이처포레,은행동,632,2018-07-16,29,14,1594,N,경기도 시흥시 은행동 632,938647.914229085,1937950.14102906
3,시흥시,LH네이처하임,조남동,720,2016-07-19,24,8,592,Y,경기도 시흥시 조남동 720,943753.023193479,1931405.64854684
4,시흥시,LH리벤티움,조남동,678,2017-07-04,21,12,944,N,경기도 시흥시 조남동 678,942958.242729263,1930742.445851


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,시흥시,LH가온수플,조남동,663,2018-07-03,25,10,678,N,경기도 시흥시 조남동 663,942863.200408787,1931018.66333105,126.854649,37.376484
1,시흥시,LH네이처뷰,장현동,170-2,2019-09-09,25,6,620,N,경기도 시흥시 장현동 170-2,937896.12810012,1931732.41392525,126.798490,37.382598
2,시흥시,LH네이처포레,은행동,632,2018-07-16,29,14,1594,N,경기도 시흥시 은행동 632,938647.914229085,1937950.14102906,126.806465,37.438689
3,시흥시,LH네이처하임,조남동,720,2016-07-19,24,8,592,Y,경기도 시흥시 조남동 720,943753.023193479,1931405.64854684,126.864669,37.380027
4,시흥시,LH리벤티움,조남동,678,2017-07-04,21,12,944,N,경기도 시흥시 조남동 678,942958.242729263,1930742.445851,126.855743,37.374001


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [15]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [16]:
#5 seoul university 
map

In [17]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_siheung.html")

In [18]:
apt_df.to_csv("apt_siheung.csv", index = False)